In [1]:
import numpy as np
import pandas as pd
import pymysql
import pyodbc
import os
from dotenv import load_dotenv
import csv

### 전처리

#### 컴포즈

In [2]:
# 파일 불러오기
compose_df = pd.read_csv('./compose/compose.csv', encoding='cp949')

# 컬럼 이름 변경
compose_df.rename(columns={
    compose_df.columns[2] : 'francise_addr',
    compose_df.columns[3] : 'francise_tel'
    }, inplace= True)


compose_df_ver2 = compose_df[
    compose_df.columns[1:]
]


In [3]:
# 결측치를 처리하자 !

# 지점명
print(
    compose_df_ver2[compose_df_ver2[compose_df_ver2.columns[0]].isna() == True].shape[0]
    )

# 주소
print(
    compose_df_ver2[compose_df_ver2[compose_df_ver2.columns[1]].isna() == True].shape[0]
    )

# 전화번호
print(
    compose_df_ver2[compose_df_ver2[compose_df_ver2.columns[2]].isna() == True].shape[0]
    )
    


0
0
490


In [4]:
#NaN을 NULL로 처리(왜냐하면 MySQL은 NaN을 처리할 수 없기에 진행 필요)
compose_df_ver2.replace({np.nan : 'NULL'}, inplace= True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\449856285.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compose_df_ver2.replace({np.nan : 'NULL'}, inplace= True)


In [6]:
compose_df_ver2[compose_df_ver2[compose_df_ver2.columns[2]] == 'NULL']

,francise_name,francise_addr,francise_tel
1,경기광주밀목점,경기 광주시 중앙로 327,NULL
2,수원시청역점,경기 수원시 팔달구 효원로 278 파비오더씨타 109호,NULL
8,당정역점,경기 군포시 고산로211번길 31 1층 107호,NULL
18,마천중앙시장점,서울 송파구 마천로41길 1,NULL
24,과천RF사옥점,경기 과천시 과천대로12길 110 제1동 1층 105호,NULL
...,...,...,...
2466,진해용원점,경남 창원시 진해구 용원로7번길 9 101호,NULL
2473,송도힐스테이트점,"인천 연수구 컨벤시아대로 90 (송도동, 인천송도힐스테이트)",NULL
2484,좌동점,부산 해운대구 좌동순환로 174 106호,NULL
2510,해운대미포점,부산 해운대구 달맞이길62번길 29 삼삼횟집 1층,NULL


In [7]:
compose_df_ver2['cafe_gubun'] = 1 #1은 컴포즈

In [8]:
compose_df_ver2

,francise_name,francise_addr,francise_tel,cafe_gubun
0,일산만자고개점,경기 고양시 일산서구 덕이로 225-3 2층 203호,031-911-0667,1
1,경기광주밀목점,경기 광주시 중앙로 327,NULL,1
2,수원시청역점,경기 수원시 팔달구 효원로 278 파비오더씨타 109호,NULL,1
3,인천청학용담공원점,인천 연수구 용담로 44 102호,032-221-0032,1
4,양산경동스마트점,경남 양산시 신덕계로 34 상가동 102호,055-383-8382,1
...,...,...,...,...
2524,정관1호점,부산 기장군 정관읍 정관로 385 (모전리),051-817-4668,1
2525,부산교대점,부산 연제구 교대로 18 (거제동),051-506-0355,1
2526,컴포즈커피 동서대점,부산 사상구 가야대로366번길 13 (주례동),051-315-8650,1
2527,동아대점,"부산 사하구 낙동대로516번길 48 (하단동, 하단에덴아파트)",051-294-2638,1


#### 메가 커피

In [9]:
mega_df = pd.read_csv('mega/mega.csv', encoding= 'cp949')
mega_df.rename(columns={
                   mega_df.columns[2] : 'francise_addr',
                   mega_df.columns[3] : 'francise_tel'
                }, inplace = True
                )
mega_df_ver2 = mega_df[mega_df.columns[1:]]

In [10]:
# 결측치를 처리하자

# 컬럼별 결측치 확인
# 지점
print(mega_df_ver2[mega_df_ver2[mega_df_ver2.columns[0]].isna() == True].shape[0])

# 주소
print(mega_df_ver2[mega_df_ver2[mega_df_ver2.columns[1]].isna() == True].shape[0])

# 전화번호
print(mega_df_ver2[mega_df_ver2[mega_df_ver2.columns[2]].isna() == True].shape[0])

0
0
0


In [11]:
# flag 컬럼을 추가하자!
mega_df_ver2['cafe_gubun'] = 2 #2는 메가

mega_df_ver2

,francise_name,francise_addr,francise_tel,cafe_gubun
0,동서울대점,"경기 성남시 수정구 복정로 73 (복정동), 1층",031-722-0111,2
1,동서울터미널점,"서울 광진구 강변역로4길 10 (구의동, 강변역지너스타워)",02-454-1003,2
2,북서울꿈의숲점,"서울 성북구 돌곶이로 220 (장위동, 꿈의숲코오롱하늘채아파트), 상가104동 107호",02-912-8982,2
3,삼육서울병원점,"서울특별시 동대문구 망우로 78, 1층(휘경동)",02-2213-2322,2
4,서서울호수공원점,서울 양천구 남부순환로 432 (신월동),02-2690-6011,2
...,...,...,...,...
3085,제주함덕해수욕장점,"제주특별자치도 제주시 조천읍 조함해안로 528, 1층",064-782-0647,2
3086,제주혁신도시점,"제주특별자치도 서귀포시 대청로 34 , (강정동)1층",064-738-0477,2
3087,서귀포여중점,제주특별자치도 서귀포시 일주동로 8840 (서홍동),064-762-9777,2
3088,서귀포효돈점,"제주특별자치도 서귀포시 일주동로 8088 , 1층(하효동)",064-767-7778,2


#### 빽다방

In [12]:
paik_df = pd.read_csv('paik/paik.csv')

In [13]:
paik_df.rename(columns={
    paik_df.columns[1] : 'francise_name',
    paik_df.columns[2] : 'francise_addr',
    paik_df.columns[3] : 'francise_tel',
    }
    , inplace = True)

In [14]:
paik_df_ver2 = paik_df[paik_df.columns[1:]]

In [15]:
# 결측치 확인

#지점
print(
    paik_df_ver2[paik_df_ver2[paik_df_ver2.columns[0]].isna() == True]\
    .shape[0]
    )

print(
    paik_df_ver2[paik_df_ver2[paik_df_ver2.columns[1]].isna() == True]\
    .shape[0]
    )

print(
    paik_df_ver2[paik_df_ver2[paik_df_ver2.columns[2]].isna() == True]\
    .shape[0]
    )


0
0
76


In [16]:
#결측치 NULL 처리
paik_df_ver2.replace({np.nan : 'NULL'}, inplace = True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\494130795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paik_df_ver2.replace({np.nan : 'NULL'}, inplace = True)


In [17]:
#결측치 재확인
paik_df_ver2[paik_df_ver2[paik_df_ver2.columns[2]].isna() == True]

,francise_name,francise_addr,francise_tel


In [18]:
# 컬럼 추가
paik_df_ver2['cafe_gubun'] = 3 # 3은 빽다방

In [19]:
paik_df_ver2

,francise_name,francise_addr,francise_tel,cafe_gubun
0,둔촌동역점,서울시 강동구 양재대로 1367,02-478-1205,3
1,김제모다아울렛점,전북특별자치도 김제시 용지면 콩쥐팥쥐로 844,063-770-8989,3
2,시흥배곧한라점,경기도 시흥시 서울대학로 172-20,070-4647-1013,3
3,종로정동사거리점,서울특별시 종로구 새문안로 46,NULL,3
4,대전대흥자이점,대전광역시 중구 충무로107번길 122,070-5056-4401,3
...,...,...,...,...
1699,신대방삼거리점,서울특별시 동작구 국사봉1길 28,02-825-5077,3
1700,충남대병원점,대전광역시 중구 계룡로920번길 36,042-220-5901,3
1701,태평로점,"서울시 중구 세종대로 82, 1층",02-318-3323,3
1702,구미상모점,"경상북도 구미시 상사서로 77, 1층",054-465-5460,3


### 데이터 삽입

#### MSSQL 연결

In [21]:
load_dotenv()

config = {
    'server' : os.getenv('host'),
    'database' : os.getenv('database'),
    'username' : os.getenv('user'),
    'password' : os.getenv('password'),
    'driver' : 'ODBC Driver 17 for SQL Server',
    'port' : os.getenv('db_port')
}

conn_str = f"DRIVER={config['driver']};SERVER={config['server']};DATABASE={config['database']};UID={config['username']};PWD={config['password']}"
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


#### 컴포즈 커피

In [24]:
for _, i in compose_df_ver2.iterrows():
    name = i[0]
    addr = i[1]
    tel = i[2]
    gubun = i[3]
    cursor.execute('''
                   INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                   VALUES(?, ?, ?, ?)
                   ''', (name, addr, tel, gubun)
                   )
    
conn.commit()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\21878654.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\21878654.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\21878654.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[2]
C:\Users\Public\Documents\ESTsof

#### 메가 커피

In [25]:
for _, i in mega_df_ver2.iterrows():
    name = i[0]
    addr = i[1]
    tel = i[2]
    gubun = i[3]
    cursor.execute('''
                INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                values (?, ?, ?, ?)
                 ''', (name, addr, tel, gubun)
                )
    
conn.commit()
    


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\1547063657.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\1547063657.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\1547063657.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[2]
C:\Users\Public\Documents\

#### 빽다방

In [26]:
for _, i in paik_df_ver2.iterrows():
    name = i[0]
    addr = i[1]
    tel = i[2]
    gubun = i[3]

    cursor.execute('''
                   INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                   values(?, ?, ?, ?)
                   ''', (name, addr, tel, gubun)
                   )
    
conn.commit()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\4029981904.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\4029981904.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24256\4029981904.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[2]
C:\Users\Public\Documents\

#### MySQL 연결

In [ ]:
# DB 연결하자
load_dotenv()

config = {
    'user' : os.getenv('user'),
    'password' : os.getenv('password'),
    'host' : 'localhost',
    'database' : 'cheapcoffee'
}

conn = pymysql.connect(**config)
cursor = conn.cursor()
cursor

#### 컴포즈 커피 삽입

In [ ]:
for i in compose_df_ver2.iterrows():
    name = i[1][0]
    addr = i[1][1]
    tel = i[1][2]
    gubun = i[1][3]
    cursor.execute('''
                   INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                   VALUES(%s, %s, %s, %s)
                   ''', (name, addr, tel, gubun)
                   )
    
conn.commit()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1261172119.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[1][0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1261172119.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1][1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1261172119.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[1][2]
C:\Users\Public\D

#### 메가 커피 삽입

In [ ]:
for i in mega_df_ver2.iterrows():
    name = i[1][0]
    addr = i[1][1]
    tel = i[1][2]
    gubun = i[1][3]
    cursor.execute('''
                INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                values (%s, %s, %s, %s)
                 ''', (name, addr, tel, gubun)
                )
    
conn.commit()
    


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\432902159.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[1][0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\432902159.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1][1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\432902159.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[1][2]
C:\Users\Public\Docu

#### 빽다방

In [108]:
for i in paik_df_ver2.iterrows():
    name = i[1][0]
    addr = i[1][1]
    tel = i[1][2]
    gubun = i[1][3]

    cursor.execute('''
                   INSERT INTO tbCafeList(francise_name, francise_addr, francise_tel, cafe_gubun)
                   values(%s,%s,%s,%s)
                   ''', (name, addr, tel, gubun)
                   )
    
conn.commit()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1586445102.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = i[1][0]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1586445102.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  addr = i[1][1]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16092\1586445102.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tel = i[1][2]
C:\Users\Public\D

In [105]:
paik_df_ver2

,francise_name,francise_addr,francise_tel,cafe_gubun
0,둔촌동역점,서울시 강동구 양재대로 1367,02-478-1205,3
1,김제모다아울렛점,전북특별자치도 김제시 용지면 콩쥐팥쥐로 844,063-770-8989,3
2,시흥배곧한라점,경기도 시흥시 서울대학로 172-20,070-4647-1013,3
3,종로정동사거리점,서울특별시 종로구 새문안로 46,NULL,3
4,대전대흥자이점,대전광역시 중구 충무로107번길 122,070-5056-4401,3
...,...,...,...,...
1699,신대방삼거리점,서울특별시 동작구 국사봉1길 28,02-825-5077,3
1700,충남대병원점,대전광역시 중구 계룡로920번길 36,042-220-5901,3
1701,태평로점,"서울시 중구 세종대로 82, 1층",02-318-3323,3
1702,구미상모점,"경상북도 구미시 상사서로 77, 1층",054-465-5460,3
